In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import duckdb
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_path = './train.parquet'
test_path = './test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,199037785,O=C(Nc1ccc(Cl)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NCC1CCC(C(F)F)CC1,CN1CCN(Cc2cccc(N)c2)CC1,CN1CCN(Cc2cccc(Nc3nc(NCC4CCC(C(F)F)CC4)nc(Nc4c...,HSA,0
1,289601389,O=C(O)[C@H]1CCCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Nc1ccc(Br)c(F)n1,Nc1ncnc(Cl)c1Cl,O=C(N[Dy])[C@H]1CCCCN1c1nc(Nc2ccc(Br)c(F)n2)nc...,HSA,0
2,218868708,O=C(Nc1ccnc(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Nc1ccc2nccnc2c1,COC(=O)c1cncc(N)c1,COC(=O)c1cncc(Nc2nc(Nc3ccnc(C(=O)N[Dy])c3)nc(N...,BRD4,0
3,217496601,O=C(Nc1cccnc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cn1ccc2cc(N)ccc21,Nc1ccncc1Cl,Cn1ccc2cc(Nc3nc(Nc4ccncc4Cl)nc(Nc4cccnc4C(=O)N...,BRD4,0
4,18389458,CC(C)CC(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Nc1ccc2c(c1)CNCC2,Cc1ccc(C#N)cc1N,Cc1ccc(C#N)cc1Nc1nc(Nc2ccc3c(c2)CNCC3)nc(NC(CC...,HSA,0


# preprocess

In [4]:
#code from GPT
def compute_ecfp(smiles_string):
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is not None:  # Ensure the molecule could be parsed
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        return list(fp)
    return [0]*2048  # Return a zero vector if the molecule could not be parsed

# Apply the function to the 'molecule_smiles' column
df['ecfp'] = df['molecule_smiles'].apply(compute_ecfp)

#would run about 3 min

KeyboardInterrupt: 

In [9]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,ecfp
0,283014445,O=C(O)[C@@H]1CSCN1C(=O)OCC1c2ccccc2-c2ccccc21,Nc1c2ccccc2nc2ccccc12,COC(=O)c1cc(OC)c(OC)cc1N,COC(=O)c1cc(OC)c(OC)cc1Nc1nc(Nc2c3ccccc3nc3ccc...,HSA,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,271489147,O=C(O)C[C@H](Cc1ccc([N+](=O)[O-])cc1)NC(=O)OCC...,COC(CN)CC(N)=O.Cl,Nc1ncnc2[nH]ncc12,COC(CNc1nc(Nc2ncnc3[nH]ncc23)nc(N[C@H](CC(=O)N...,HSA,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,31826420,COc1cc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c...,COc1c(F)ccc(F)c1CN.Cl,Cl.Cl.NCc1cc(Br)cc2cccnc12,COc1cc(Nc2nc(NCc3c(F)ccc(F)c3OC)nc(NCc3cc(Br)c...,sEH,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,289213389,O=C(O)[C@H]1CCCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1cccc(-n2ccnn2)c1,COc1ccccc1-c1nnc(N)s1,COc1ccccc1-c1nnc(Nc2nc(NCc3cccc(-n4ccnn4)c3)nc...,BRD4,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,254204808,O=C(O)C[C@@H](Cc1cccs1)NC(=O)OCC1c2ccccc2-c2cc...,Cc1cc(N)cc(F)c1,CC1CCCC(CN)O1,Cc1cc(F)cc(Nc2nc(NCC3CCCC(C)O3)nc(N[C@@H](CC(=...,BRD4,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [5]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))
# Convert numpy array to tensor
protein_features = torch.tensor(protein_onehot, dtype=torch.float)

In [11]:
from rdkit import Chem
import torch
from torch_geometric.data import Data

def molecule_to_graph(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None

    atoms = molecule.GetAtoms()
    bonds = molecule.GetBonds()

    # Node features: Atomic number
    node_features = [atom.GetAtomicNum() for atom in atoms]
    node_features = torch.tensor(node_features, dtype=torch.float).unsqueeze(1)  # Unsqueeze for feature dimension
    
    # Edge indices and features
    edge_index = []
    edge_features = []
    for bond in bonds:
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index.append((start, end))
        edge_index.append((end, start))  # Because graphs are undirected
        edge_features.append(bond.GetBondTypeAsDouble())  # Bond type as double

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_features = torch.tensor(edge_features, dtype=torch.float)

    #combined_features = torch.tensor(protein_onehot.tolist(), dtype=torch.float)
    #    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, u=combined_features)

    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)


In [6]:
def molecule_to_graph(molecule_smiles, protein_feature):
    molecule = Chem.MolFromSmiles(molecule_smiles)
    atoms = molecule.GetAtoms()
    bonds = molecule.GetBonds()

    node_features = torch.tensor([atom.GetAtomicNum() for atom in atoms], dtype=torch.float).unsqueeze(1)
    edge_index = torch.tensor([(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()) for bond in bonds], dtype=torch.long).t().contiguous()

    return Data(x=node_features, edge_index=edge_index, protein_features=protein_feature)


In [7]:
# Assuming 'df' has a column 'molecule_smiles' containing SMILES strings
data_list = [molecule_to_graph(smiles, protein_features[i]) for i, smiles in enumerate(df['molecule_smiles'])]



In [33]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader

# Model definition
class MoleculeProteinGNN(torch.nn.Module):
    def __init__(self, num_node_features, num_protein_features, num_classes):
        super(MoleculeProteinGNN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.conv2 = GCNConv(128, 64)
        self.fc1 = torch.nn.Linear(64 + num_protein_features, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        protein_features = data.protein_features
        if protein_features.dim() == 1:
            protein_features = protein_features.unsqueeze(0)  # Ensure it is 2D
        x = torch.cat((x, protein_features), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Example Data preparation
# Assume data_list is correctly prepared and contains Data objects with .x, .edge_index, .batch, and .protein_features
train_loader = DataLoader(data_list[:int(0.8 * len(data_list))], batch_size=32, shuffle=True)
test_loader = DataLoader(data_list[int(0.8 * len(data_list)):], batch_size=32, shuffle=False)

# Model instantiation
model = MoleculeProteinGNN(num_node_features=1, num_protein_features=3, num_classes=1)  # Adjust as per actual feature counts

# Loss and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for data in train_loader:
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, data.y.float().unsqueeze(1))  # Match output dimensions
    loss.backward()
    optimizer.step()

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        output = model(data)
        predictions = (torch.sigmoid(output) > 0.5).float()  # Convert logits to binary predictions
        correct += (predictions == data.y.unsqueeze(1)).sum().item()  # Ensure dimensions match
        total += data.y.size(0)

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [27]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 32)
        self.fc = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Average pooling
        x = self.fc(x)
        return torch.sigmoid(x)

In [28]:
data_list = [Data(...)]  # Create a list of Data objects from your dataset
loader = DataLoader(data_list, batch_size=32, shuffle=True)

# Model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

# Training loop
model.train()
for epoch in range(200):
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}, Loss: {loss.item()}')

NameError: name 'num_node_features' is not defined

In [ ]:
import os

# Process the test.parquet file chunk by chunk
test_file = './test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(smiles_to_ecfp)

    # One-hot encode the protein_name
    protein_onehot =  OneHotEncoder(sparse_output=False).transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))